<a href="https://colab.research.google.com/github/MarcosVeniciu/Producao-de-cafe-MG/blob/main/Processamento_do_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Carregar dataframes

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
indicadores_IBGE = pd.read_excel('/content/Indicadores IBGE por Municipio - 1995 a 2023.xlsx')
indicadores_climaticos = pd.read_csv('/content/climate_por_municipio_1990_2024.csv')
georeferenciamento = pd.read_excel('/content/municipios_georeferenciamento.xlsx')

In [ ]:
indicadores_IBGE.info()

In [ ]:
indicadores_climaticos.info()

In [ ]:
georeferenciamento.info()

# Dataset V5 - Produção (1995-2023) - Serie temporal


**Features**
- Ano
- latitude
- longitude
- altitude
- Área Colhida (mil hectares) (área plantada)
- ~~Valor da produção (Mil Reais)~~
- precipitacao (mm)
- temperatura minima (ºC)
- temperatura maxima (ºC)
- Mesorregião

Estimando **produção em Toneladas**.

## 2. Merge das bases espacial (meso_id) e temporal (Ano)

In [ ]:
# Calculando a média anual por municipio
clima_anual = indicadores_climaticos.groupby(
    ['municipio_id', 'Ano'],
    as_index=False
).agg({
    'Mesorregião_id': 'first',      # Mantém o ID da mesorregião associada ao município
    'municipio_nome': 'first',      # Mantém o nome do município (assume que é único por município_id)
    'prec_mm': 'mean',              # Média anual da precipitação
    'tmax_°C': 'mean',              # Média anual da temperatura máxima
    'tmin_°C': 'mean'               # Média anual da temperatura mínima
})

clima_anual.rename(columns={
  'municipio_id': 'Municipio_id',
  'prec_mm': 'precipitacao (mm)',
  'municipio_nome': 'Municipio',
  'tmax_°C': 'temperatura maxima (ºC)',
  'tmin_°C': 'temperatura minima (ºC)'
}, inplace=True)

clima_anual.head(5)

In [ ]:
# Lista de Produtos: 'Café (em grão) Total', 'Café (em grão) Arábica', 'Café (em grão) Canephora'
indicadores_IBGE_filtrado = indicadores_IBGE[indicadores_IBGE['Produto']=="Café (em grão) Total"].copy()
indicadores_IBGE_filtrado.drop(columns=['Produto'], inplace=True)

# Renomear as colunas do dataframe georeferenciamento
georeferenciamento_renomeado = georeferenciamento.rename(columns={
    'municipio_id': 'Municipio_id',
    'lat': 'latitude',
    'lon': 'longitude',
    'alt': 'altitude'
})

# Selecionar as colunas desejadas de cada dataframe
colunas_clima = ['Municipio_id', 'Ano', 'precipitacao (mm)', 'temperatura maxima (ºC)', 'temperatura minima (ºC)']
colunas_ibge = [
    'Mesorregião_id', 'Mesorregião', 'Municipio_id', 'Municipio', 'Ano',
    'Área destinada à colheita (Hectares)', 'Área colhida (Hectares)',
    'Quantidade produzida (Toneladas)', 'Rendimento médio da produção (Quilogramas por Hectare)',
    'Valor da produção (Mil Reais)'
]
colunas_geo = ['Municipio_id', 'latitude', 'longitude', 'altitude']

# Filtrar os dataframes pelas colunas desejadas
clima_filtrado = clima_anual[colunas_clima]
ibge_filtrado = indicadores_IBGE_filtrado[colunas_ibge]
geo_filtrado = georeferenciamento_renomeado[colunas_geo]

In [ ]:
# Mesclar clima_filtrado e ibge_filtrado usando Municipio_id e Ano como chaves
df_mesclado = pd.merge(clima_filtrado, ibge_filtrado, on=['Municipio_id', 'Ano'], how='inner')

# Mesclar o resultado com geo_filtrado usando Municipio_id como chave
df_merge = pd.merge(df_mesclado, geo_filtrado, on='Municipio_id', how='inner')

df_merge.head(1)

In [ ]:
# 1) Cria uma cópia para não alterar o original
df_escalado = df_merge.copy()

# Garante que as colunas numéricas estejam no tipo correto
df_escalado['Área colhida (Hectares)'] = pd.to_numeric(df_escalado['Área colhida (Hectares)'], errors='coerce')
df_escalado['Quantidade produzida (Toneladas)'] = pd.to_numeric(df_escalado['Quantidade produzida (Toneladas)'], errors='coerce')

df_escalado.head(1)

## 3. Filtrar período de interesse (1995–2023)

In [ ]:
df_periodo = df_escalado[(df_escalado['Ano'] >= 1995) & (df_escalado['Ano'] <= 2023)].copy()

## 4. Criar variável-alvo: produtividade em Toneldas por hectare

In [ ]:
df_target = df_periodo.copy()
df_target['target'] = df_target['Quantidade produzida (Toneladas)']

## 5. Remoção de outliers usando critério de desvio‑padrão (±3σ)

In [ ]:
df_target[["target"]].describe()

In [ ]:
print(len(df_target))

valor = 3
y = df_target['target']
mean_y, std_y = y.mean(), y.std()
mask = (y >= mean_y - valor*std_y) & (y <= mean_y + valor*std_y)
df_outliers = df_target.loc[mask].reset_index(drop=True)
print(len(df_outliers))

In [ ]:
df_outliers[["target"]].describe()

## 6. Imputação de faltantes: aqui usamos mediana (poderia ser média ou KNN)

In [ ]:
# Antes da imputação
dados_faltantes_antes = df_outliers.isnull().sum()
print("Dados faltantes antes da imputação:\n", dados_faltantes_antes)

In [ ]:
df_outliers.fillna(df_outliers.median(numeric_only=True), inplace=True)

## 7. Organizar serie temporal

In [ ]:
import pandas as pd

def count_time_series_examples(df: pd.DataFrame,
                               id_col: str = 'Municipio_id',
                               year_col: str = 'Ano',
                               min_context: int = 2,
                               horizon: int = 1) -> pd.DataFrame:
    """
    Para cada tamanho de janela de contexto a partir de `min_context`, computa quantos pares
    (janela de contexto -> horizonte) são possíveis em cada série do DataFrame.

    Passos:
    1. Agrupa por `id_col` e ordena por `year_col`.
    2. Identifica segmentos contíguos (sem saltos de anos).
    3. Seleciona o maior segmento contíguo para cada série.
    4. Para cada contexto c, soma todos os exemplos possíveis: max(0, L - c - horizon + 1).
    5. Para contextos em que nenhum exemplo seja possível, para o cálculo.

    Retorna um DataFrame com colunas:
        context: tamanho da janela de contexto
        examples: número total de pares treino->teste disponíveis
    """
    # 1. Prepara o DataFrame
    df_sorted = df[[id_col, year_col]].drop_duplicates().copy()
    df_sorted = df_sorted.sort_values([id_col, year_col])

    # 2. Identifica segmentos contíguos
    # Calcula a diferença de anos dentro de cada grupo
    df_sorted['year_diff'] = df_sorted.groupby(id_col)[year_col].diff().fillna(1)
    # Marca início de segmento onde diff > 1
    df_sorted['segment_id'] = (df_sorted['year_diff'] > 1).cumsum()

    # 3. Computa tamanho de cada segmento e seleciona o maior por série
    segments = (
        df_sorted
        .groupby([id_col, 'segment_id'])
        .agg(start_year=(year_col, 'min'),
             end_year=(year_col, 'max'),
             length=(year_col, 'count'))
        .reset_index()
    )
    # Seleciona o segmento de maior comprimento para cada série
    max_segments = (
        segments
        .sort_values(['length'], ascending=False)
        .drop_duplicates(id_col, keep='first')
    )

    # 4. Varredura de contextos
    results = []
    c = min_context
    while True:
        # Para cada segmento, se length >= c + horizon, gera (length - c - horizon + 1) exemplos
        max_segments['examples'] = max_segments['length'].apply(
            lambda L: max(0, L - c - horizon + 1)
        )
        total_examples = int(max_segments['examples'].sum())
        if total_examples <= 0:
            break
        results.append({'context': c, 'examples': total_examples})
        c += 1

    return pd.DataFrame(results)

count_time_series_examples(df_outliers)

In [ ]:
def filter_series_by_context(df: pd.DataFrame,
                              context: int,
                              id_col: str = 'Municipio_id',
                              year_col: str = 'Ano',
                              horizon: int = 1) -> pd.DataFrame:
    """
    Filtra o DataFrame mantendo apenas as séries cujo maior segmento contíguo
    gera pelo menos 2 pares treino->teste para o dado tamanho de contexto.

    Parâmetros:
    - df: DataFrame completo com colunas de identificação e ano.
    - context: tamanho da janela de contexto (número de anos).
    - id_col: nome da coluna de identificador de série.
    - year_col: nome da coluna de ano.
    - horizon: horizonte de previsão (padrão 1 ano).

    Retorna:
    - DataFrame filtrado, contendo apenas os registros dos municípios válidos.
    """
    # 1) Obtenha apenas as combinações únicas (id, ano) já ordenadas
    df_sorted = (
        df[[id_col, year_col]]
        .drop_duplicates()
        .sort_values([id_col, year_col])
    )

    # 2) Identifique quebras de sequência de anos
    df_sorted['year_diff'] = (
        df_sorted
        .groupby(id_col)[year_col]
        .diff()
        .fillna(1)
    )
    df_sorted['segment_id'] = (df_sorted['year_diff'] > 1).cumsum()

    # 3) Calcule início, fim e comprimento de cada segmento
    segments = (
        df_sorted
        .groupby([id_col, 'segment_id'])
        .agg(
            start_year=(year_col, 'min'),
            end_year=(year_col, 'max'),
            length=(year_col, 'count')
        )
        .reset_index()
    )

    # 4) Para cada série (cada id_col), selecione o segmento mais longo
    max_seg = (
        segments
        .sort_values([id_col, 'length'], ascending=[True, False])
        .drop_duplicates(subset=id_col, keep='first')
        .reset_index(drop=True)
    )

    # 5) Filtre apenas os segmentos que dão pelo menos 2 pares treino->teste:
    #    comprimento >= context + horizon + 1
    min_length = context + horizon + 1
    valid = max_seg[max_seg['length'] >= min_length]

    # 6) Reúna do DataFrame original apenas os anos desses segmentos válidos
    dfs = []
    for _, row in valid.iterrows():
        mid = row[id_col]
        anos_validos = range(int(row['start_year']), int(row['end_year']) + 1)
        mask = (df[id_col] == mid) & (df[year_col].isin(anos_validos))
        dfs.append(df.loc[mask])

    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        return df.iloc[0:0]  # vazio, mesmas colunas

# Exemplo de uso:
janela_contexto = 2
df_filtrado = filter_series_by_context(df_outliers, janela_contexto)

## 8. Seleção de features

Mantemos variáveis numéricas e dummies de mesorregião; descartamos IDs se não necessários.

In [ ]:
num_features = [
    'target',
    'Municipio',
    'Ano',
    'latitude',
    'longitude',
    'altitude',
    'precipitacao (mm)',
    'temperatura minima (ºC)',
    'temperatura maxima (ºC)'
]
cat_features = [
  'Mesorregião'
]

df_features = df_filtrado[num_features + cat_features].copy()

## 9. Divisão temporal treino/teste (anos ≤ 2009 → treino, ≥ 2010 → teste)

In [ ]:
horizon = 1
year_col = 'Ano'
id_col = 'Municipio'

df = df_features.copy()
df['split'] = 'train'
k = janela_contexto + horizon
# Para cada série, marca como test as últimas k observações
def mark_test(sub):
    if len(sub) <= k:
        return pd.Series(['test'] * len(sub), index=sub.index)
    test_idx = sub.nlargest(k, year_col).index
    splits = pd.Series('train', index=sub.index)
    splits.loc[test_idx] = 'test'
    return splits
df['split'] = df.groupby(id_col).apply(lambda g: mark_test(g), include_groups=False).reset_index(level=0, drop=True)

print(df['split'].value_counts())

In [ ]:
df[df.Municipio == "Abadia dos Dourados - MG"]

In [ ]:
df.to_csv('dataset_v5.csv', index=False)
df.info()

# Dataset V4 - Produtividade (1995-2023) - Serie temporal


**Features**
- Ano
- latitude
- longitude
- altitude
- ~~Área Colhida (mil hectares)~~
- ~~Valor da produção (Mil Reais)~~
- precipitacao (mm)
- temperatura minima (ºC)
- temperatura maxima (ºC)
- Mesorregião

Estimando **produtividade em Toneladas/Hectares**.

## 2. Merge das bases espacial (meso_id) e temporal (Ano)

In [ ]:
# Calculando a média anual por municipio
clima_anual = indicadores_climaticos.groupby(
    ['municipio_id', 'Ano'],
    as_index=False
).agg({
    'Mesorregião_id': 'first',      # Mantém o ID da mesorregião associada ao município
    'municipio_nome': 'first',      # Mantém o nome do município (assume que é único por município_id)
    'prec_mm': 'mean',              # Média anual da precipitação
    'tmax_°C': 'mean',              # Média anual da temperatura máxima
    'tmin_°C': 'mean'               # Média anual da temperatura mínima
})

clima_anual.rename(columns={
  'municipio_id': 'Municipio_id',
  'prec_mm': 'precipitacao (mm)',
  'municipio_nome': 'Municipio',
  'tmax_°C': 'temperatura maxima (ºC)',
  'tmin_°C': 'temperatura minima (ºC)'
}, inplace=True)

clima_anual.head(5)

In [ ]:
# Lista de Produtos: 'Café (em grão) Total', 'Café (em grão) Arábica', 'Café (em grão) Canephora'
indicadores_IBGE_filtrado = indicadores_IBGE[indicadores_IBGE['Produto']=="Café (em grão) Total"].copy()
indicadores_IBGE_filtrado.drop(columns=['Produto'], inplace=True)

# Renomear as colunas do dataframe georeferenciamento
georeferenciamento_renomeado = georeferenciamento.rename(columns={
    'municipio_id': 'Municipio_id',
    'lat': 'latitude',
    'lon': 'longitude',
    'alt': 'altitude'
})

# Selecionar as colunas desejadas de cada dataframe
colunas_clima = ['Municipio_id', 'Ano', 'precipitacao (mm)', 'temperatura maxima (ºC)', 'temperatura minima (ºC)']
colunas_ibge = [
    'Mesorregião_id', 'Mesorregião', 'Municipio_id', 'Municipio', 'Ano',
    'Área destinada à colheita (Hectares)', 'Área colhida (Hectares)',
    'Quantidade produzida (Toneladas)', 'Rendimento médio da produção (Quilogramas por Hectare)',
    'Valor da produção (Mil Reais)'
]
colunas_geo = ['Municipio_id', 'latitude', 'longitude', 'altitude']

# Filtrar os dataframes pelas colunas desejadas
clima_filtrado = clima_anual[colunas_clima]
ibge_filtrado = indicadores_IBGE_filtrado[colunas_ibge]
geo_filtrado = georeferenciamento_renomeado[colunas_geo]

In [ ]:
# Mesclar clima_filtrado e ibge_filtrado usando Municipio_id e Ano como chaves
df_mesclado = pd.merge(clima_filtrado, ibge_filtrado, on=['Municipio_id', 'Ano'], how='inner')

# Mesclar o resultado com geo_filtrado usando Municipio_id como chave
df_merge = pd.merge(df_mesclado, geo_filtrado, on='Municipio_id', how='inner')

df_merge.head(1)

In [ ]:
# 1) Cria uma cópia para não alterar o original
df_escalado = df_merge.copy()

# Garante que as colunas numéricas estejam no tipo correto
df_escalado['Área colhida (Hectares)'] = pd.to_numeric(df_escalado['Área colhida (Hectares)'], errors='coerce')
df_escalado['Quantidade produzida (Toneladas)'] = pd.to_numeric(df_escalado['Quantidade produzida (Toneladas)'], errors='coerce')

df_escalado.head(1)

## 3. Filtrar período de interesse (1995–2023)

In [ ]:
df_periodo = df_escalado[(df_escalado['Ano'] >= 1995) & (df_escalado['Ano'] <= 2023)].copy()

## 4. Criar variável-alvo: produtividade em Toneldas por hectare

In [ ]:
df_target = df_periodo.copy()
df_target['target'] = df_target['Quantidade produzida (Toneladas)'] / df_target['Área colhida (Hectares)'] # produtividade em toneladas por hectare

## 5. Remoção de outliers usando critério de desvio‑padrão (±3σ)

In [ ]:
df_target[["target"]].describe()

In [ ]:
print(len(df_target))

valor = 3
y = df_target['target']
mean_y, std_y = y.mean(), y.std()
mask = (y >= mean_y - valor*std_y) & (y <= mean_y + valor*std_y)
df_outliers = df_target.loc[mask].reset_index(drop=True)
print(len(df_outliers))

In [ ]:
df_outliers[["target"]].describe()

## 6. Imputação de faltantes: aqui usamos mediana (poderia ser média ou KNN)

In [ ]:
# Antes da imputação
dados_faltantes_antes = df_outliers.isnull().sum()
print("Dados faltantes antes da imputação:\n", dados_faltantes_antes)

In [ ]:
df_outliers.fillna(df_outliers.median(numeric_only=True), inplace=True)

## 7. Organizar serie temporal

In [ ]:
import pandas as pd

def count_time_series_examples(df: pd.DataFrame,
                               id_col: str = 'Municipio_id',
                               year_col: str = 'Ano',
                               min_context: int = 2,
                               horizon: int = 1) -> pd.DataFrame:
    """
    Para cada tamanho de janela de contexto a partir de `min_context`, computa quantos pares
    (janela de contexto -> horizonte) são possíveis em cada série do DataFrame.

    Passos:
    1. Agrupa por `id_col` e ordena por `year_col`.
    2. Identifica segmentos contíguos (sem saltos de anos).
    3. Seleciona o maior segmento contíguo para cada série.
    4. Para cada contexto c, soma todos os exemplos possíveis: max(0, L - c - horizon + 1).
    5. Para contextos em que nenhum exemplo seja possível, para o cálculo.

    Retorna um DataFrame com colunas:
        context: tamanho da janela de contexto
        examples: número total de pares treino->teste disponíveis
    """
    # 1. Prepara o DataFrame
    df_sorted = df[[id_col, year_col]].drop_duplicates().copy()
    df_sorted = df_sorted.sort_values([id_col, year_col])

    # 2. Identifica segmentos contíguos
    # Calcula a diferença de anos dentro de cada grupo
    df_sorted['year_diff'] = df_sorted.groupby(id_col)[year_col].diff().fillna(1)
    # Marca início de segmento onde diff > 1
    df_sorted['segment_id'] = (df_sorted['year_diff'] > 1).cumsum()

    # 3. Computa tamanho de cada segmento e seleciona o maior por série
    segments = (
        df_sorted
        .groupby([id_col, 'segment_id'])
        .agg(start_year=(year_col, 'min'),
             end_year=(year_col, 'max'),
             length=(year_col, 'count'))
        .reset_index()
    )
    # Seleciona o segmento de maior comprimento para cada série
    max_segments = (
        segments
        .sort_values(['length'], ascending=False)
        .drop_duplicates(id_col, keep='first')
    )

    # 4. Varredura de contextos
    results = []
    c = min_context
    while True:
        # Para cada segmento, se length >= c + horizon, gera (length - c - horizon + 1) exemplos
        max_segments['examples'] = max_segments['length'].apply(
            lambda L: max(0, L - c - horizon + 1)
        )
        total_examples = int(max_segments['examples'].sum())
        if total_examples <= 0:
            break
        results.append({'context': c, 'examples': total_examples})
        c += 1

    return pd.DataFrame(results)

count_time_series_examples(df_outliers)

In [ ]:
def filter_series_by_context(df: pd.DataFrame,
                              context: int,
                              id_col: str = 'Municipio_id',
                              year_col: str = 'Ano',
                              horizon: int = 1) -> pd.DataFrame:
    """
    Filtra o DataFrame mantendo apenas as séries cujo maior segmento contíguo
    gera pelo menos 2 pares treino->teste para o dado tamanho de contexto.

    Parâmetros:
    - df: DataFrame completo com colunas de identificação e ano.
    - context: tamanho da janela de contexto (número de anos).
    - id_col: nome da coluna de identificador de série.
    - year_col: nome da coluna de ano.
    - horizon: horizonte de previsão (padrão 1 ano).

    Retorna:
    - DataFrame filtrado, contendo apenas os registros dos municípios válidos.
    """
    # 1) Obtenha apenas as combinações únicas (id, ano) já ordenadas
    df_sorted = (
        df[[id_col, year_col]]
        .drop_duplicates()
        .sort_values([id_col, year_col])
    )

    # 2) Identifique quebras de sequência de anos
    df_sorted['year_diff'] = (
        df_sorted
        .groupby(id_col)[year_col]
        .diff()
        .fillna(1)
    )
    df_sorted['segment_id'] = (df_sorted['year_diff'] > 1).cumsum()

    # 3) Calcule início, fim e comprimento de cada segmento
    segments = (
        df_sorted
        .groupby([id_col, 'segment_id'])
        .agg(
            start_year=(year_col, 'min'),
            end_year=(year_col, 'max'),
            length=(year_col, 'count')
        )
        .reset_index()
    )

    # 4) Para cada série (cada id_col), selecione o segmento mais longo
    max_seg = (
        segments
        .sort_values([id_col, 'length'], ascending=[True, False])
        .drop_duplicates(subset=id_col, keep='first')
        .reset_index(drop=True)
    )

    # 5) Filtre apenas os segmentos que dão pelo menos 2 pares treino->teste:
    #    comprimento >= context + horizon + 1
    min_length = context + horizon + 1
    valid = max_seg[max_seg['length'] >= min_length]

    # 6) Reúna do DataFrame original apenas os anos desses segmentos válidos
    dfs = []
    for _, row in valid.iterrows():
        mid = row[id_col]
        anos_validos = range(int(row['start_year']), int(row['end_year']) + 1)
        mask = (df[id_col] == mid) & (df[year_col].isin(anos_validos))
        dfs.append(df.loc[mask])

    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        return df.iloc[0:0]  # vazio, mesmas colunas

# Exemplo de uso:
janela_contexto = 2
df_filtrado = filter_series_by_context(df_outliers, janela_contexto)

## 8. Seleção de features

Mantemos variáveis numéricas e dummies de mesorregião; descartamos IDs se não necessários.

In [ ]:
num_features = [
    'target',
    'Municipio',
    'Ano',
    'latitude',
    'longitude',
    'altitude',
    'precipitacao (mm)',
    'temperatura minima (ºC)',
    'temperatura maxima (ºC)'
]
cat_features = [
  'Mesorregião'
]

df_features = df_filtrado[num_features + cat_features].copy()

## 9. Divisão temporal treino/teste (anos ≤ 2009 → treino, ≥ 2010 → teste)

In [ ]:
horizon = 1
year_col = 'Ano'
id_col = 'Municipio'

df = df_features.copy()
df['split'] = 'train'
k = janela_contexto + horizon
# Para cada série, marca como test as últimas k observações
def mark_test(sub):
    if len(sub) <= k:
        return pd.Series(['test'] * len(sub), index=sub.index)
    test_idx = sub.nlargest(k, year_col).index
    splits = pd.Series('train', index=sub.index)
    splits.loc[test_idx] = 'test'
    return splits
df['split'] = df.groupby(id_col).apply(lambda g: mark_test(g), include_groups=False).reset_index(level=0, drop=True)

print(df['split'].value_counts())

In [ ]:
df[df.Municipio == "Abadia dos Dourados - MG"]

In [ ]:
df.to_csv('dataset_v4.csv', index=False)
df.info()

# Dataset V3 - Produtividade

Nesta versão, ao inves de avaliar a quantidade de toneladas de café produzida, será avaliada a produtividade, que é a quantidade produzida por área (toneladas/hectares)

**Features**
- Ano
- latitude
- longitude
- altitude
- ~~Área Colhida (mil hectares)~~
- ~~Valor da produção (Mil Reais)~~
- precipitacao (mm)
- temperatura minima (ºC)
- temperatura maxima (ºC)
- Mesorregião

Estimando **produtividade em Toneladas/Hectares**.

## 2. Merge das bases espacial (meso_id) e temporal (Ano)

In [ ]:
# Calculando a média anual por municipio
clima_anual = indicadores_climaticos.groupby(
    ['municipio_id', 'Ano'],
    as_index=False
).agg({
    'Mesorregião_id': 'first',      # Mantém o ID da mesorregião associada ao município
    'municipio_nome': 'first',      # Mantém o nome do município (assume que é único por município_id)
    'prec_mm': 'mean',              # Média anual da precipitação
    'tmax_°C': 'mean',              # Média anual da temperatura máxima
    'tmin_°C': 'mean'               # Média anual da temperatura mínima
})

clima_anual.rename(columns={
  'municipio_id': 'Municipio_id',
  'prec_mm': 'precipitacao (mm)',
  'municipio_nome': 'Municipio',
  'tmax_°C': 'temperatura maxima (ºC)',
  'tmin_°C': 'temperatura minima (ºC)'
}, inplace=True)

clima_anual.head(5)

In [ ]:
# Lista de Produtos: 'Café (em grão) Total', 'Café (em grão) Arábica', 'Café (em grão) Canephora'
indicadores_IBGE_filtrado = indicadores_IBGE[indicadores_IBGE['Produto']=="Café (em grão) Total"].copy()
indicadores_IBGE_filtrado.drop(columns=['Produto'], inplace=True)

# Renomear as colunas do dataframe georeferenciamento
georeferenciamento_renomeado = georeferenciamento.rename(columns={
    'municipio_id': 'Municipio_id',
    'lat': 'latitude',
    'lon': 'longitude',
    'alt': 'altitude'
})

# Selecionar as colunas desejadas de cada dataframe
colunas_clima = ['Municipio_id', 'Ano', 'precipitacao (mm)', 'temperatura maxima (ºC)', 'temperatura minima (ºC)']
colunas_ibge = [
    'Mesorregião_id', 'Mesorregião', 'Municipio_id', 'Municipio', 'Ano',
    'Área destinada à colheita (Hectares)', 'Área colhida (Hectares)',
    'Quantidade produzida (Toneladas)', 'Rendimento médio da produção (Quilogramas por Hectare)',
    'Valor da produção (Mil Reais)'
]
colunas_geo = ['Municipio_id', 'latitude', 'longitude', 'altitude']

# Filtrar os dataframes pelas colunas desejadas
clima_filtrado = clima_anual[colunas_clima]
ibge_filtrado = indicadores_IBGE_filtrado[colunas_ibge]
geo_filtrado = georeferenciamento_renomeado[colunas_geo]

In [ ]:
# Mesclar clima_filtrado e ibge_filtrado usando Municipio_id e Ano como chaves
df_mesclado = pd.merge(clima_filtrado, ibge_filtrado, on=['Municipio_id', 'Ano'], how='inner')

# Mesclar o resultado com geo_filtrado usando Municipio_id como chave
df_merge = pd.merge(df_mesclado, geo_filtrado, on='Municipio_id', how='inner')

df_merge.head(2)

In [ ]:
# 1) Cria uma cópia para não alterar o original
df_escalado = df_merge.copy()

# Garante que as colunas numéricas estejam no tipo correto
df_escalado['Área colhida (Hectares)'] = pd.to_numeric(df_escalado['Área colhida (Hectares)'], errors='coerce')
df_escalado['Quantidade produzida (Toneladas)'] = pd.to_numeric(df_escalado['Quantidade produzida (Toneladas)'], errors='coerce')

df_escalado.head(2)

## 3. Filtrar período de interesse (1995–2023)

In [ ]:
df_periodo = df_escalado[(df_escalado['Ano'] >= 1995) & (df_escalado['Ano'] <= 2023)].copy()

## 4. Criar variável-alvo: produtividade em Kilos por hectare

In [ ]:
df_target = df_periodo.copy()
df_target['target'] = df_target['Quantidade produzida (Toneladas)'] / df_target['Área colhida (Hectares)'] # produtividade em toneladas por hectare

## 5. Remoção de outliers usando critério de desvio‑padrão (±3σ)

In [ ]:
df_target[["target"]].describe()

In [ ]:
print(len(df_target))

valor = 3
y = df_target['target']
mean_y, std_y = y.mean(), y.std()
mask = (y >= mean_y - valor*std_y) & (y <= mean_y + valor*std_y)
df_outliers = df_target.loc[mask].reset_index(drop=True)
print(len(df_outliers))

In [ ]:
df_outliers[["target"]].describe()

## 6. Imputação de faltantes: aqui usamos mediana (poderia ser média ou KNN)

In [ ]:
# Antes da imputação
dados_faltantes_antes = df_outliers.isnull().sum()
print("Dados faltantes antes da imputação:\n", dados_faltantes_antes)

In [ ]:
df_outliers.fillna(df_outliers.median(numeric_only=True), inplace=True)

## 7. Seleção de features

Mantemos variáveis numéricas e dummies de mesorregião; descartamos IDs se não necessários.

In [ ]:
num_features = [
    'target',
    'Municipio',
    'Ano',
    'latitude',
    'longitude',
    'altitude',
    'precipitacao (mm)',
    'temperatura minima (ºC)',
    'temperatura maxima (ºC)'
]
cat_features = [
  'Mesorregião'
]
dataset_final = df_outliers[num_features + cat_features].copy()

## 8. Divisão temporal treino/teste (anos ≤ 2015 → treino, ≥ 2016 → teste)

In [ ]:
dataset_final['split'] = pd.Series(dtype='str')  # Inicializa a coluna com tipo string
dataset_final.loc[dataset_final['Ano'] <= 2015, 'split'] = 'train'
dataset_final.loc[dataset_final['Ano'] >= 2016, 'split'] = 'test'

# Verifica o resultado
print(dataset_final['split'].value_counts())

In [ ]:
dataset_final.to_csv('dataset_v3.csv', index=False)
dataset_final.info()

# Dataset V2 - Produção

Nesta versão apeas o valor da produção é removido em relação ao anterior.

**Features**
- Ano
- latitude
- longitude
- altitude
- Área Colhida (mil hectares)
- ~~Valor da produção (Mil Reais)~~
- precipitacao (mm)
- temperatura minima (ºC)
- temperatura maxima (ºC)
- Mesorregião

Estimando **quantidade produzida em toneladas**.

## 2. Merge das bases espacial (meso_id) e temporal (Ano)

In [ ]:
# Calculando a média anual por municipio
clima_anual = indicadores_climaticos.groupby(
    ['municipio_id', 'Ano'],
    as_index=False
).agg({
    'Mesorregião_id': 'first',      # Mantém o ID da mesorregião associada ao município
    'municipio_nome': 'first',      # Mantém o nome do município (assume que é único por município_id)
    'prec_mm': 'mean',              # Média anual da precipitação
    'tmax_°C': 'mean',              # Média anual da temperatura máxima
    'tmin_°C': 'mean'               # Média anual da temperatura mínima
})

clima_anual.rename(columns={
  'municipio_id': 'Municipio_id',
  'prec_mm': 'precipitacao (mm)',
  'municipio_nome': 'Municipio',
  'tmax_°C': 'temperatura maxima (ºC)',
  'tmin_°C': 'temperatura minima (ºC)'
}, inplace=True)

clima_anual.head(5)

In [ ]:
# Lista de Produtos: 'Café (em grão) Total', 'Café (em grão) Arábica', 'Café (em grão) Canephora'
indicadores_IBGE_filtrado = indicadores_IBGE[indicadores_IBGE['Produto']=="Café (em grão) Total"].copy()
indicadores_IBGE_filtrado.drop(columns=['Produto'], inplace=True)

# Renomear as colunas do dataframe georeferenciamento
georeferenciamento_renomeado = georeferenciamento.rename(columns={
    'municipio_id': 'Municipio_id',
    'lat': 'latitude',
    'lon': 'longitude',
    'alt': 'altitude'
})

# Selecionar as colunas desejadas de cada dataframe
colunas_clima = ['Municipio_id', 'Ano', 'precipitacao (mm)', 'temperatura maxima (ºC)', 'temperatura minima (ºC)']
colunas_ibge = [
    'Mesorregião_id', 'Mesorregião', 'Municipio_id', 'Municipio', 'Ano',
    'Área destinada à colheita (Hectares)', 'Área colhida (Hectares)',
    'Quantidade produzida (Toneladas)', 'Rendimento médio da produção (Quilogramas por Hectare)',
    'Valor da produção (Mil Reais)'
]
colunas_geo = ['Municipio_id', 'latitude', 'longitude', 'altitude']

# Filtrar os dataframes pelas colunas desejadas
clima_filtrado = clima_anual[colunas_clima]
ibge_filtrado = indicadores_IBGE_filtrado[colunas_ibge]
geo_filtrado = georeferenciamento_renomeado[colunas_geo]

In [ ]:
# Mesclar clima_filtrado e ibge_filtrado usando Municipio_id e Ano como chaves
df_mesclado = pd.merge(clima_filtrado, ibge_filtrado, on=['Municipio_id', 'Ano'], how='inner')

# Mesclar o resultado com geo_filtrado usando Municipio_id como chave
df_merge = pd.merge(df_mesclado, geo_filtrado, on='Municipio_id', how='inner')

df_merge.head(2)

In [ ]:
# 1) Cria uma cópia para não alterar o original
df_escalado = df_merge.copy()

# Garante que as colunas numéricas estejam no tipo correto
df_escalado['Área colhida (Hectares)'] = pd.to_numeric(df_escalado['Área colhida (Hectares)'], errors='coerce')
df_escalado['Quantidade produzida (Toneladas)'] = pd.to_numeric(df_escalado['Quantidade produzida (Toneladas)'], errors='coerce')

df_escalado.head(2)

## 3. Filtrar período de interesse (1995–2023)

In [ ]:
df_periodo = df_escalado[(df_escalado['Ano'] >= 1995) & (df_escalado['Ano'] <= 2023)].copy()

## 4. Criar variável-alvo: Quantidade produzida em TONELADAS

In [ ]:
coluna_alvo = 'Quantidade produzida (Toneladas)'
df_target = df_periodo.rename(columns={coluna_alvo: 'target'})

## 5. Remoção de outliers usando critério de desvio‑padrão (±3σ)

In [ ]:
df_target[["target"]].describe()

In [ ]:
print(len(df_target))

valor = 3
y = df_target['target']
mean_y, std_y = y.mean(), y.std()
mask = (y >= mean_y - valor*std_y) & (y <= mean_y + valor*std_y)
df_outliers = df_target.loc[mask].reset_index(drop=True)
print(len(df_outliers))

In [ ]:
df_outliers[["target"]].describe()

## 6. Imputação de faltantes: aqui usamos mediana (poderia ser média ou KNN)

In [ ]:
# Antes da imputação
dados_faltantes_antes = df_outliers.isnull().sum()
print("Dados faltantes antes da imputação:\n", dados_faltantes_antes)

In [ ]:
df_outliers.fillna(df_outliers.median(numeric_only=True), inplace=True)

## 7. Seleção de features

Mantemos variáveis numéricas e dummies de mesorregião; descartamos IDs se não necessários.

In [ ]:
num_features = [
    'target',
    'Municipio',
    'Ano',
    'latitude',
    'longitude',
    'altitude',
    'Área colhida (Hectares)',
    'precipitacao (mm)',
    'temperatura minima (ºC)',
    'temperatura maxima (ºC)'
]
cat_features = [
  'Mesorregião'
]

## 8. Divisão temporal treino/teste (anos ≤ 2015 → treino, ≥ 2016 → teste)

In [ ]:
dataset_final = df_outliers[num_features + cat_features].copy()

dataset_final['split'] = pd.Series(dtype='str')  # Inicializa a coluna com tipo string
dataset_final.loc[dataset_final['Ano'] <= 2015, 'split'] = 'train'
dataset_final.loc[dataset_final['Ano'] >= 2016, 'split'] = 'test'

# Verifica o resultado
print(dataset_final['split'].value_counts())

In [ ]:
dataset_final.to_csv('dataset_v2.csv', index=False)
dataset_final.info()

# Dataset V1 - Todos os atributos

**Features**
- Ano
- latitude
- longitude
- altitude
- Área Colhida (hectares)
- Valor da produção (Mil Reais)
- precipitacao (mm)
- temperatura minima (ºC)
- temperatura maxima (ºC)
- Mesorregião

Estimando **quantidade produzida**.

## 2. Merge das bases espacial (meso_id) e temporal (Ano)

In [ ]:
# Calculando a média anual por municipio
clima_anual = indicadores_climaticos.groupby(
    ['municipio_id', 'Ano'],
    as_index=False
).agg({
    'Mesorregião_id': 'first',      # Mantém o ID da mesorregião associada ao município
    'municipio_nome': 'first',      # Mantém o nome do município (assume que é único por município_id)
    'prec_mm': 'mean',              # Média anual da precipitação
    'tmax_°C': 'mean',              # Média anual da temperatura máxima
    'tmin_°C': 'mean'               # Média anual da temperatura mínima
})

clima_anual.rename(columns={
  'municipio_id': 'Municipio_id',
  'prec_mm': 'precipitacao (mm)',
  'municipio_nome': 'Municipio',
  'tmax_°C': 'temperatura maxima (ºC)',
  'tmin_°C': 'temperatura minima (ºC)'
}, inplace=True)

clima_anual.head(5)

In [ ]:
# Lista de Produtos: 'Café (em grão) Total', 'Café (em grão) Arábica', 'Café (em grão) Canephora'
indicadores_IBGE_filtrado = indicadores_IBGE[indicadores_IBGE['Produto']=="Café (em grão) Total"].copy()
indicadores_IBGE_filtrado.drop(columns=['Produto'], inplace=True)

# Renomear as colunas do dataframe georeferenciamento
georeferenciamento_renomeado = georeferenciamento.rename(columns={
    'municipio_id': 'Municipio_id',
    'lat': 'latitude',
    'lon': 'longitude',
    'alt': 'altitude'
})

# Selecionar as colunas desejadas de cada dataframe
colunas_clima = ['Municipio_id', 'Ano', 'precipitacao (mm)', 'temperatura maxima (ºC)', 'temperatura minima (ºC)']
colunas_ibge = [
    'Mesorregião_id', 'Mesorregião', 'Municipio_id', 'Municipio', 'Ano',
    'Área destinada à colheita (Hectares)', 'Área colhida (Hectares)',
    'Quantidade produzida (Toneladas)', 'Rendimento médio da produção (Quilogramas por Hectare)',
    'Valor da produção (Mil Reais)'
]
colunas_geo = ['Municipio_id', 'latitude', 'longitude', 'altitude']

# Filtrar os dataframes pelas colunas desejadas
clima_filtrado = clima_anual[colunas_clima]
ibge_filtrado = indicadores_IBGE_filtrado[colunas_ibge]
geo_filtrado = georeferenciamento_renomeado[colunas_geo]

In [ ]:
# Mesclar clima_filtrado e ibge_filtrado usando Municipio_id e Ano como chaves
df_mesclado = pd.merge(clima_filtrado, ibge_filtrado, on=['Municipio_id', 'Ano'], how='inner')

# Mesclar o resultado com geo_filtrado usando Municipio_id como chave
df_merge = pd.merge(df_mesclado, geo_filtrado, on='Municipio_id', how='inner')

df_merge.head(2)

In [ ]:
# 1) Cria uma cópia para não alterar o original
df_escalado = df_merge.copy()

# Garante que as colunas numéricas estejam no tipo correto
df_escalado['Área colhida (Hectares)'] = pd.to_numeric(df_escalado['Área colhida (Hectares)'], errors='coerce')
df_escalado['Quantidade produzida (Toneladas)'] = pd.to_numeric(df_escalado['Quantidade produzida (Toneladas)'], errors='coerce')

df_escalado.head(2)

## 3. Filtrar período de interesse (1995–2023)

In [ ]:
df_periodo = df_escalado[(df_escalado['Ano'] >= 1995) & (df_escalado['Ano'] <= 2023)].copy()

## 4. Criar variável-alvo: Quantidade produzida em MIL TONELADAS

In [ ]:
coluna_alvo = 'Quantidade produzida (Toneladas)'
df_target = df_periodo.rename(columns={coluna_alvo: 'target'})

## 5. Remoção de outliers usando critério de desvio‑padrão (±3σ)

In [ ]:
df_target[["target"]].describe()

In [ ]:
print(len(df_target))

valor = 3
y = df_target['target']
mean_y, std_y = y.mean(), y.std()
mask = (y >= mean_y - valor*std_y) & (y <= mean_y + valor*std_y)
df_outliers = df_target.loc[mask].reset_index(drop=True)
print(len(df_outliers))

In [ ]:
df_outliers[["target"]].describe()

## 6. Imputação de faltantes: aqui usamos mediana (poderia ser média ou KNN)

In [ ]:
# Antes da imputação
dados_faltantes_antes = df_outliers.isnull().sum()
print("Dados faltantes antes da imputação:\n", dados_faltantes_antes)

In [ ]:
df_outliers.fillna(df_outliers.median(numeric_only=True), inplace=True)

## 7. Seleção de features

Mantemos variáveis numéricas e dummies de mesorregião; descartamos IDs se não necessários. \
Comentário: meso_id, lat, lon, altitude_m não entram no modelo após one-hot

In [ ]:
num_features = [
    'target',
    'Municipio',
    'Ano',
    'latitude',
    'longitude',
    'altitude',
    'Área colhida (Hectares)',
    'Valor da produção (Mil Reais)',
    'precipitacao (mm)',
    'temperatura minima (ºC)',
    'temperatura maxima (ºC)'
]
cat_features = [
  'Mesorregião'
]

## 8. Divisão temporal treino/teste (anos ≤ 2009 → treino, ≥ 2010 → teste)

In [ ]:
dataset_final = df_outliers[num_features + cat_features].copy()

dataset_final['split'] = pd.Series(dtype='str')  # Inicializa a coluna com tipo string
dataset_final.loc[dataset_final['Ano'] <= 2015, 'split'] = 'train'
dataset_final.loc[dataset_final['Ano'] >= 2016, 'split'] = 'test'

# Verifica o resultado
print(dataset_final['split'].value_counts())

In [ ]:
dataset_final.to_csv('dataset_v1.csv', index=False)
dataset_final.info()